In [ ]:
import os
import cv2
import numpy as np
import moviepy.video.io.ImageSequenceClip
from moviepy.editor import VideoFileClip
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM

time = 31
channels = 33

for codec in ['libvpx']:
    out_folder = f'outputs/moviepy/{codec}'
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    for ci in range(channels):
        vid_imgs = []
        for ti in range(time):
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            vid_imgs.append(im)
        clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(vid_imgs, fps=24)
        clip.write_videofile(f"{out_folder}/{ci+1}.webm", audio=False, preset='veryslow', logger=None, codec=codec)
    psnrs = []
    ssims = []
    for ci in range(channels):
        clip = VideoFileClip(f"{out_folder}/{ci+1}.webm")
        for ti, frames in enumerate(clip.iter_frames()):
            if ti >= time:
                continue
            gray_frames = cv2.cvtColor(frames, cv2.COLOR_RGB2GRAY)
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            psnrs.append(PSNR(gray_frames, im))
            ssims.append(SSIM(gray_frames, im))
    with open(f"{out_folder}/metrics.txt",'w') as f:
        f.write(f"MEAN PSNR:{np.array(psnrs).mean()}\n")
        f.write(f"MEAN SSIM:{np.array(ssims).mean()}\n")